In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import math
import time
import os
import scipy as sp
from scipy import stats # statistics

service_key = 'hDaSIy4ntwfjGKQRK49VW8xYuR5Wi8HUCCr2pnL2wrGjq675JjRPOxb6e%2F9Xtg7N94DRG37oQr30uZY1JS6b3g%3D%3D'

    itemF009 윗몸말아올리기(회)
    itemF010 반복점프(회)
    itemF012 앉아윗몸앞으로굽히기(cm)
    itemF013 일리노이(초)
    itemF014 체공시간(초)
    itemF015 협응력시간(초)
    itemF016 협응력실수횟수(회)
    itemF017 협응력계산결과값(초)
    itemF019 교차윗몸일으키기(회)
    itemF020 왕복오래달리기(회)
    itemF021 10M 4회 왕복달리기(초)
    itemF022 제자리 멀리뛰기(cm)
    itemF023 의자에앉았다일어서기(회)
    itemF024 6분걷기(m)
    itemF025 2분제자리걷기(회)
    itemF026 의자에앉아 3M표적 돌아오기(초)
    itemF027 8자보행(초)
    itemF030 왕복오래달리기(출력)(VO₂max)
    itemF031 트레드밀_안정시(bpm)
    itemF032 트레드밀_3분(bpm)
    itemF033 트레드밀_6분(bpm)
    itemF034 트레드밀9분(bpm)
    itemF035 트레드밀(출력)(VO₂max)
    itemF036 스텝검사회복시 심박수(bpm)
    itemF037 스텝검사(출력)(VO₂max)
    itemF040 전신반응(초)
    itemF041 체공시간(초)

# API로 체력 데이터 불러오기

In [ ]:
# 추출하고 싶은 체력측정 번호 골라담기
nums=['09'] # '9'는 한자리 숫자이기 때문에 '09'로 미리 list에 담아두고
for i in range(10,42): # 나머지 번호 append
    if i not in [11,18,28,29,38,39]:
        nums.append(str(i))
print(nums)

['09', '10', '12', '13', '14', '15', '16', '17', '19', '20', '21', '22', '23', '24', '25', '26', '27', '30', '31', '32', '33', '34', '35', '36', '37', '40', '41']


In [ ]:
def monthly_df_merger(service_key, testYm):   # 서비스키, '201801' 형태의 년월
    base_url = 'http://www.kspo.or.kr/openapi/service/nfaTestInfoService/getNfaTestRsltList'
    
    '''1. 월별로 API로 불러내기 전에 data 개수가 몇 개 있는지 먼저 파악하는 과정
    (데이터 개수를 totalCount 변수로 저장해 1page에 전체 데이터 개수를 배치하고 1page 전체를 스크래핑)'''
    
    test_url = base_url + '?' + 'serviceKey=' + service_key + '&pageNo=' + '1' +\
                '&numOfRows=' + '100' + '&testYm=' + testYm
    test_response = requests.get(test_url).content
    test_soup = BeautifulSoup(test_response, 'lxml-xml')
    totalCount = test_soup.find('totalCount').get_text()
    print(totalCount)
    
    
    
    '''2. HTTP Response 얻고, HTML 얻기'''
    url = base_url + '?' + 'serviceKey=' + service_key + '&pageNo=' + '1' +\
        '&numOfRows=' + totalCount + '&testYm=' + testYm
    print(url)
    response = requests.get(url)
    print(response)
    
    response = requests.get(url).content
    soup = BeautifulSoup(response, 'lxml-xml')

    item = soup.find_all('item')
    
    
    '''3. 'item' 태그 꺼내 각 사람의 체력 데이터 dict에 모으기 -> 한달치를 list에 append -> dataframe'''
    collect_dict_list=[]   # 한 달 모든 사람들의 dict값을 모을 list
    for i in range(len(item)):
        monthly_person_data={}   # 한 달 각 사람들의 체력 데이터를 모을 dict
        for num in nums:
            try:
                monthly_person_data['itemF0{}'.format(num)] = item[i].find('itemF0{}'.format(num)).get_text()
            except:
                pass
        collect_dict_list.append(monthly_person_data)

    monthly_total_data = pd.DataFrame.from_dict(collect_dict_list)   # dict를 담은 list를 바로 dict로 변환
    return monthly_total_data

In [ ]:
for year in range(2019,2022):   # 2018~2019년까지 반복해서 추출
    testYm=[]   # monthly_df_merger 함수 사용을 위해 '201801'~'202012'까지 만들기
    for month in range(1,13):
        if month < 10:
            testYm.append(str(year) + '0' + str(month))
        else:
            testYm.append(str(year) + str(month))
    print(testYm)
    
    monthly_list=[]   # list에 1월 ~ 12월까지 모든 사람들의 체력 측정 데이터 모으기
    for idx, ym in enumerate(testYm, start=1):
        print('12달 중 {}번째 달입니다'.format(idx))
        healthdf_2018 = monthly_list.append(monthly_df_merger(service_key, ym))
    
    healthdf = pd.concat(monthly_list)   # 모아진 list들을 pd.concat으로 다 연결시켜 dataframe으로 만들기
    healthdf.to_csv('healthdf_{}.csv'.format(year), index=False)   # 저장

['201901', '201902', '201903', '201904', '201905', '201906', '201907', '201908', '201909', '201910', '201911', '201912']
12달 중 1번째 달입니다
6187
http://www.kspo.or.kr/openapi/service/nfaTestInfoService/getNfaTestRsltList?serviceKey=hDaSIy4ntwfjGKQRK49VW8xYuR5Wi8HUCCr2pnL2wrGjq675JjRPOxb6e%2F9Xtg7N94DRG37oQr30uZY1JS6b3g%3D%3D&pageNo=1&numOfRows=6187&testYm=201901
<Response [200]>
12달 중 2번째 달입니다
5447
http://www.kspo.or.kr/openapi/service/nfaTestInfoService/getNfaTestRsltList?serviceKey=hDaSIy4ntwfjGKQRK49VW8xYuR5Wi8HUCCr2pnL2wrGjq675JjRPOxb6e%2F9Xtg7N94DRG37oQr30uZY1JS6b3g%3D%3D&pageNo=1&numOfRows=5447&testYm=201902
<Response [200]>
12달 중 3번째 달입니다
27804
http://www.kspo.or.kr/openapi/service/nfaTestInfoService/getNfaTestRsltList?serviceKey=hDaSIy4ntwfjGKQRK49VW8xYuR5Wi8HUCCr2pnL2wrGjq675JjRPOxb6e%2F9Xtg7N94DRG37oQr30uZY1JS6b3g%3D%3D&pageNo=1&numOfRows=27804&testYm=201903
<Response [200]>
12달 중 4번째 달입니다
46988
http://www.kspo.or.kr/openapi/service/nfaTestInfoService/getNfaTestRsltList?serviceKe

# 코로나 전/후 데이터로 정리

In [ ]:
'''
os.chdir('C:/Users/user/Python_MLDL/Project/Semi-Project')
os.getcwd()
'''

'C:\\Users\\user\\Python_MLDL\\Project\\Semi-Project'

In [ ]:
for i in range(2018,2022):
    globals()['healthdf_{}'.format(str(i))] = pd.read_csv('healthdf_{}.csv'.format(str(i)))

In [ ]:
healthdf_2018

,itemF009,itemF010,itemF012,itemF013,itemF014,itemF015,itemF016,itemF017,itemF019,itemF020,...,itemF030,itemF031,itemF032,itemF033,itemF034,itemF035,itemF036,itemF037,itemF040,itemF041
0,-,-,3.3,-,-,-,-,-,16,8,...,25.7,-,-,-,-,-,-,-,-,-
1,-,-,5,-,-,-,-,-,42,33,...,37.8,-,-,-,-,-,-,-,-,-
2,-,-,14.5,-,-,-,-,-,40,33,...,38.6,-,-,-,-,-,-,-,-,-
3,-,-,10,-,-,-,-,-,16,10,...,31.3,-,-,-,-,-,-,-,-,-
4,-,-,15.4,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267396,80,-,15,20.88,0.471,71.821,11,72.327,-,69,...,47.7,-,-,-,-,-,-,-,-,-
267397,-,-,13.8,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
267398,42,-,10.5,17.11,0.680,44.123,10,44.583,-,45,...,39,-,-,-,-,-,-,-,-,-
267399,-,-,-11.5,-,-,-,-,-,45,15,...,36.3,-,-,-,-,-,-,-,-,-


In [ ]:
healthdf_2019

,itemF009,itemF010,itemF012,itemF013,itemF014,itemF015,itemF016,itemF017,itemF019,itemF020,...,itemF030,itemF031,itemF032,itemF033,itemF034,itemF035,itemF036,itemF037,itemF040,itemF041
0,-,-,14.5,-,-,-,-,-,57,54,...,44.7,-,-,-,-,-,-,-,-,-
1,-,-,13,-,-,-,-,-,66,84,...,54,-,-,-,-,-,-,-,-,-
2,-,-,13,-,-,-,-,-,59,114,...,59.4,-,-,-,-,-,-,-,-,-
3,-,-,22,-,-,-,-,-,53,83,...,51.9,-,-,-,-,-,-,-,-,-
4,-,-,16,-,-,-,-,-,55,80,...,51,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303766,-,-,21.7,-,-,-,-,-,18,-,...,-,64,119,153,-,31.6,-,-,-,-
303767,-,-,3.4,-,-,-,-,-,53,51,...,45.8,-,-,-,-,-,-,-,-,-
303768,-,-,4.7,-,-,-,-,-,49,55,...,46.3,-,-,-,-,-,-,-,-,-
303769,-,58,13.5,17.16,0.674,45.460,17,46.242,-,68,...,52.8,-,-,-,-,-,-,-,-,-


In [ ]:
healthdf_2020

,itemF009,itemF010,itemF012,itemF013,itemF014,itemF015,itemF016,itemF017,itemF019,itemF020,...,itemF030,itemF031,itemF032,itemF033,itemF034,itemF035,itemF036,itemF037,itemF040,itemF041
0,-,-,8.2,-,-,-,-,-,57,60,...,47.2,-,-,-,-,-,-,-,-,-
1,-,-,10.5,-,-,-,-,-,50,35,...,39.8,-,-,-,-,-,-,-,-,-
2,-,-,20.5,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
3,-,-,25.4,-,-,-,-,-,24,25,...,34,-,-,-,-,-,-,-,-,-
4,-,-,21.0,-,-,-,-,-,40,31,...,36.4,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79766,-,-,18.5,-,-,-,-,-,23,-,...,-,-,-,-,-,-,119,37.2,0.382,-
79767,-,-,22.8,-,-,-,-,-,44,-,...,-,-,-,-,-,-,135,45,0.298,-
79768,-,-,5,-,-,-,-,-,46,-,...,-,-,-,-,-,-,67,49.1,0.313,-
79769,-,-,15.8,-,-,-,-,-,29,-,...,-,-,-,-,-,-,126,34.4,0.390,-


In [ ]:
healthdf_2021

,itemF009,itemF010,itemF012,itemF013,itemF014,itemF015,itemF016,itemF017,itemF019,itemF020,...,itemF030,itemF031,itemF032,itemF033,itemF034,itemF035,itemF036,itemF037,itemF040,itemF041
0,-,-,16.5,-,-,-,-,-,22,-,...,-,-,-,-,-,-,106,35.7,0.475,-
1,-,-,8.9,-,-,-,-,-,32,-,...,-,85,112,137,164,32.4,-,-,0.41,-
2,-,-,9.4,-,-,-,-,-,62,-,...,-,77,80,131,165,41.6,-,-,0.362,-
3,-,-,13.3,-,-,-,-,-,52,-,...,-,-,-,-,-,-,110,44.3,0.352,-
4,-,-,12.4,-,-,-,-,-,55,-,...,-,-,-,-,-,-,114,45,0.287,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161808,-,-,4.2,-,-,-,-,-,11,-,...,-,-,-,-,-,-,120,29.3,0.391,-
161809,-,50,8.6,21.76,0.590,41.137,10,41.597,-,-,...,-,70,114,126,-,47.5,-,-,-,-
161810,-,57,17.5,18.02,0.614,41.673,9,42.087,-,-,...,-,105,120,138,-,48.7,-,-,-,-
161811,-,-,12.4,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-


In [ ]:
# 코로나 이전 (@018, 2019) ; 코로나 이후 (2020, 2021)로 데이터프레임 합치기
healthdf_before = pd.concat([healthdf_2018, healthdf_2019])
healthdf_after = pd.concat([healthdf_2020, healthdf_2021])

In [ ]:
healthdf_before.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 571172 entries, 0 to 303770
Data columns (total 27 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   itemF009  571172 non-null  object
 1   itemF010  571172 non-null  object
 2   itemF012  571172 non-null  object
 3   itemF013  571172 non-null  object
 4   itemF014  571171 non-null  object
 5   itemF015  571172 non-null  object
 6   itemF016  571172 non-null  object
 7   itemF017  571172 non-null  object
 8   itemF019  571172 non-null  object
 9   itemF020  571172 non-null  object
 10  itemF021  571172 non-null  object
 11  itemF022  571172 non-null  object
 12  itemF023  571172 non-null  object
 13  itemF024  571172 non-null  object
 14  itemF025  571172 non-null  object
 15  itemF026  571172 non-null  object
 16  itemF027  571171 non-null  object
 17  itemF030  571172 non-null  object
 18  itemF031  571172 non-null  object
 19  itemF032  571172 non-null  object
 20  itemF033  571172 non-null 

In [ ]:
healthdf_before.describe()

,itemF009,itemF010,itemF012,itemF013,itemF014,itemF015,itemF016,itemF017,itemF019,itemF020,...,itemF030,itemF031,itemF032,itemF033,itemF034,itemF035,itemF036,itemF037,itemF040,itemF041
count,571172,571172,571172,571172,571171,571172,571172,571172,571172,571172,...,571172,571172,571172,571172,571172,571171,571171,571172,571172,571172
unique,218,165,1523,2644,741,40753,153,39385,163,351,...,1034,138,154,144,119,440,203,544,121,128
top,-,-,0,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
freq,485781,396444,8014,318301,319561,324900,324913,324901,358393,163463,...,163633,550352,550458,550505,564094,550597,529684,529755,570918,570796


In [ ]:
healthdf_after.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 241584 entries, 0 to 161812
Data columns (total 27 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   itemF009  241584 non-null  object
 1   itemF010  241584 non-null  object
 2   itemF012  241584 non-null  object
 3   itemF013  241584 non-null  object
 4   itemF014  241584 non-null  object
 5   itemF015  241584 non-null  object
 6   itemF016  241584 non-null  object
 7   itemF017  241584 non-null  object
 8   itemF019  241584 non-null  object
 9   itemF020  241584 non-null  object
 10  itemF021  241584 non-null  object
 11  itemF022  241584 non-null  object
 12  itemF023  241584 non-null  object
 13  itemF024  241584 non-null  object
 14  itemF025  241584 non-null  object
 15  itemF026  241584 non-null  object
 16  itemF027  241584 non-null  object
 17  itemF030  241584 non-null  object
 18  itemF031  241584 non-null  object
 19  itemF032  241584 non-null  object
 20  itemF033  241584 non-null 

In [ ]:
healthdf_after.describe()

,itemF009,itemF010,itemF012,itemF013,itemF014,itemF015,itemF016,itemF017,itemF019,itemF020,...,itemF030,itemF031,itemF032,itemF033,itemF034,itemF035,itemF036,itemF037,itemF040,itemF041
count,241584,241584,241584,241584,241584,241584,241584,241584,241584,241584,...,241584,241584,241584,241584,241584,241584,241584,241584,241584,241584
unique,173,98,1078,2252,1253,28525,97,29642,107,192,...,945,161,137,148,141,433,185,502,1124,637
top,-,-,0,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
freq,200496,187961,1967,160437,162438,161085,161096,161081,121909,142688,...,148310,227731,228379,228397,236509,228405,149404,149404,169914,232766


# ttest

In [ ]:
'''ERROR!!!!!'''
# 1) 불러온 type이 str이였기 때문에
# 2) ttest는 결측치 drop 해주어야 하기 때문에
for i in range(len(nums)):
    before = healthdf_before.iloc[:,i]
    after = healthdf_after.iloc[:,i]
    result = stats.ttest_ind(healthdf_before.iloc[:,i], healthdf_after.iloc[:,i])
    print(result)

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [ ]:
statistics=[]
pvalue=[]
for i in range(len(nums)):
    before = pd.to_numeric(healthdf_before.iloc[:,i].replace('-','')).dropna()
    after = pd.to_numeric(healthdf_after.iloc[:,i].replace('-','')).dropna()
    result = stats.ttest_ind(before, after)
    statistics.append(result[0])
    pvalue.append(result[1])
    print(result)

Ttest_indResult(statistic=35.299773755953275, pvalue=1.2549593382630318e-271)
Ttest_indResult(statistic=16.79354636683351, pvalue=2.970062550688103e-63)
Ttest_indResult(statistic=-23.93592132660593, pvalue=1.4336112848488092e-126)
Ttest_indResult(statistic=-0.8698353809695115, pvalue=0.38439099818690436)
Ttest_indResult(statistic=-4.330030003341398, pvalue=1.4913279286450005e-05)
Ttest_indResult(statistic=-5.613937985392889, pvalue=1.979328828807714e-08)
Ttest_indResult(statistic=-24.89788278585242, pvalue=1.0530665448825125e-136)
Ttest_indResult(statistic=-5.794489154021715, pvalue=6.859206118811306e-09)
Ttest_indResult(statistic=-100.76741786736132, pvalue=0.0)
Ttest_indResult(statistic=-20.511144682469457, pvalue=1.8691295207788762e-93)
Ttest_indResult(statistic=55.640521950478075, pvalue=0.0)
Ttest_indResult(statistic=-87.56383777841607, pvalue=0.0)
Ttest_indResult(statistic=-50.066050960427205, pvalue=0.0)
Ttest_indResult(statistic=-0.4132311115800005, pvalue=0.6794562891020062)
T

In [ ]:
healthdf_before.columns

Index(['itemF009', 'itemF010', 'itemF012', 'itemF013', 'itemF014', 'itemF015',
       'itemF016', 'itemF017', 'itemF019', 'itemF020', 'itemF021', 'itemF022',
       'itemF023', 'itemF024', 'itemF025', 'itemF026', 'itemF027', 'itemF030',
       'itemF031', 'itemF032', 'itemF033', 'itemF034', 'itemF035', 'itemF036',
       'itemF037', 'itemF040', 'itemF041'],
      dtype='object')

In [ ]:
statistics

[35.299773755953275,
 16.79354636683351,
 -23.93592132660593,
 -0.8698353809695115,
 -4.330030003341398,
 -5.613937985392889,
 -24.89788278585242,
 -5.794489154021715,
 -100.76741786736132,
 -20.511144682469457,
 55.640521950478075,
 -87.56383777841607,
 -50.066050960427205,
 -0.4132311115800005,
 -26.783085485651036,
 3.8877187048503106,
 14.146206473086131,
 161.41432384711356,
 -1.921184128592016,
 -41.3713410192768,
 -0.31233165807145824,
 -2.3301405380466154,
 -9.32873719670069,
 -101.0643652458464,
 -14.035484584766238,
 95.09130473893696,
 5.813701909818247]

In [ ]:
pd.DataFrame([pvalue, statistics]).T

,0,1
0,1.254959e-271,35.299774
1,2.970063e-63,16.793546
2,1.433611e-126,-23.935921
3,3.843910e-01,-0.869835
4,1.491328e-05,-4.330030
5,1.979329e-08,-5.613938
6,1.053067e-136,-24.897883
7,6.859206e-09,-5.794489
8,0.000000e+00,-100.767418
9,1.869130e-93,-20.511145


In [ ]:
ttest_df = pd.DataFrame([statistics,pvalue]).T
ttest_df.columns=['statistics', 'pvalue']
ttest_df.index=list(healthdf_before.columns)
ttest_df

,statistics,pvalue
itemF009,35.299774,1.254959e-271
itemF010,16.793546,2.970063e-63
itemF012,-23.935921,1.433611e-126
itemF013,-0.869835,3.843910e-01
itemF014,-4.330030,1.491328e-05
itemF015,-5.613938,1.979329e-08
itemF016,-24.897883,1.053067e-136
itemF017,-5.794489,6.859206e-09
itemF019,-100.767418,0.000000e+00
itemF020,-20.511145,1.869130e-93


# 예시

In [ ]:
monthly_df_merger(service_key, '202001')

9742
http://www.kspo.or.kr/openapi/service/nfaTestInfoService/getNfaTestRsltList?serviceKey=hDaSIy4ntwfjGKQRK49VW8xYuR5Wi8HUCCr2pnL2wrGjq675JjRPOxb6e%2F9Xtg7N94DRG37oQr30uZY1JS6b3g%3D%3D&pageNo=1&numOfRows=9742&testYm=202001
<Response [200]>
9742


,itemF009,itemF010,itemF012,itemF013,itemF014,itemF015,itemF016,itemF017,itemF019,itemF020,...,itemF030,itemF031,itemF032,itemF033,itemF034,itemF035,itemF036,itemF037,itemF040,itemF041
0,-,-,8.2,-,-,-,-,-,57,60,...,47.2,-,-,-,-,-,-,-,-,-
1,-,-,10.5,-,-,-,-,-,50,35,...,39.8,-,-,-,-,-,-,-,-,-
2,-,-,20.5,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
3,-,-,25.4,-,-,-,-,-,24,25,...,34,-,-,-,-,-,-,-,-,-
4,-,-,21.0,-,-,-,-,-,40,31,...,36.4,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,-,-,19.7,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
9738,-,-,17.4,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
9739,-,-,16.1,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
9740,-,-,13.5,-,-,-,-,-,2,11,...,28.7,-,-,-,-,-,-,-,-,-


In [ ]:
testYm=[]
year=input("년도를 입력하세요 : ")
for month in range(1,13):
    if month < 10:
        testYm.append(year + '0' + str(month))
    else:
        testYm.append(year + str(month))
print(testYm)

년도를 입력하세요 : 2018
['201801', '201802', '201803', '201804', '201805', '201806', '201807', '201808', '201809', '201810', '201811', '201812']


In [ ]:
healthdf_2018 = pd.DataFrame()
for idx, ym in enumerate(testYm, start=1):
    print('12달 중 {}번째 달입니다'.format(idx))
    healthdf_2018 = pd.concat([healthdf_2018, monthly_df_merger(service_key, ym)])

12달 중 1번째 달입니다
4337
http://www.kspo.or.kr/openapi/service/nfaTestInfoService/getNfaTestRsltList?serviceKey=hDaSIy4ntwfjGKQRK49VW8xYuR5Wi8HUCCr2pnL2wrGjq675JjRPOxb6e%2F9Xtg7N94DRG37oQr30uZY1JS6b3g%3D%3D&pageNo=1&numOfRows=4337&testYm=201801
<Response [200]>
12달 중 2번째 달입니다
5257
http://www.kspo.or.kr/openapi/service/nfaTestInfoService/getNfaTestRsltList?serviceKey=hDaSIy4ntwfjGKQRK49VW8xYuR5Wi8HUCCr2pnL2wrGjq675JjRPOxb6e%2F9Xtg7N94DRG37oQr30uZY1JS6b3g%3D%3D&pageNo=1&numOfRows=5257&testYm=201802
<Response [200]>
12달 중 3번째 달입니다
21708
http://www.kspo.or.kr/openapi/service/nfaTestInfoService/getNfaTestRsltList?serviceKey=hDaSIy4ntwfjGKQRK49VW8xYuR5Wi8HUCCr2pnL2wrGjq675JjRPOxb6e%2F9Xtg7N94DRG37oQr30uZY1JS6b3g%3D%3D&pageNo=1&numOfRows=21708&testYm=201803
<Response [200]>
12달 중 4번째 달입니다
36702
http://www.kspo.or.kr/openapi/service/nfaTestInfoService/getNfaTestRsltList?serviceKey=hDaSIy4ntwfjGKQRK49VW8xYuR5Wi8HUCCr2pnL2wrGjq675JjRPOxb6e%2F9Xtg7N94DRG37oQr30uZY1JS6b3g%3D%3D&pageNo=1&numOfRows=36702

In [ ]:
healthdf_2018

,itemF009,itemF010,itemF012,itemF013,itemF014,itemF015,itemF016,itemF017,itemF019,itemF020,...,itemF030,itemF031,itemF032,itemF033,itemF034,itemF035,itemF036,itemF037,itemF040,itemF041
0,-,-,3.3,-,-,-,-,-,16,8,...,25.7,-,-,-,-,-,-,-,-,-
1,-,-,5,-,-,-,-,-,42,33,...,37.8,-,-,-,-,-,-,-,-,-
2,-,-,14.5,-,-,-,-,-,40,33,...,38.6,-,-,-,-,-,-,-,-,-
3,-,-,10,-,-,-,-,-,16,10,...,31.3,-,-,-,-,-,-,-,-,-
4,-,-,15.4,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12637,80,-,15,20.88,0.471,71.821,11,72.327,-,69,...,47.7,-,-,-,-,-,-,-,-,-
12638,-,-,13.8,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
12639,42,-,10.5,17.11,0.680,44.123,10,44.583,-,45,...,39,-,-,-,-,-,-,-,-,-
12640,-,-,-11.5,-,-,-,-,-,45,15,...,36.3,-,-,-,-,-,-,-,-,-


In [ ]:
healthdf_2018.to_csv('healthdf_2018.csv', index=False)